In [ ]:
############################################################################################################################
### Projeto da Disciplina de PO-233                                                                                        #
### Título: Identificação de dados biométricos utilizando redes Wireless e aprendizado de máquina                          #
### Alunos:                                                                                                                #
###         Ágney Lopes Roth Ferraz                                                                                        #
###	        Alexandre Bellargus Silva da Costa                                                                             #
###	        Carlos Renato de Andrade Figueiredo                                                                            #
###	        Gioliano de Oliveira Braga                                                                                     #
###	        Paulo Ricardo Sousa Fonteles de Castro                                                                         #
###	        Wagner Comin Sonaglio                                                                                          #
############################################################################################################################

## -------------------- 01. Importação de Bibliotecas -------------------- 

Nesta célula são importadas todas as bibliotecas necessárias para o projeto. As bibliotecas estão organizadas por funcionalidade:

- **Manipulação de dados**: `numpy` e `pandas` para tratamento numérico e tabular.
- **Visualização**: `matplotlib` e `seaborn` para criação de gráficos e análises visuais.
- **Machine Learning**: modelos (como árvore de decisão), avaliação (acurácia, F1, ROC), e redução de dimensionalidade com `PCA`.
- **Validação e experimentos**: `cross_val_score` e `clone` para validação cruzada e replicação de modelos.
- **Visualização de árvores**: `graphviz` para exportar e renderizar a estrutura da árvore de decisão.
- **Utilitários**: `pathlib`, `timeit` e `importlib` para controle de caminhos, tempo de execução e recarregamento de módulos.

Essas importações preparam o ambiente para todas as etapas do aprendizado de máquina: pré-processamento, treino, avaliação e visualização.


In [ ]:
# -----------------------------
# 📦 Bibliotecas básicas
# -----------------------------
import os  # manipulação de arquivos e diretórios
import numpy as np  # operações vetoriais e matriciais
import pandas as pd  # manipulação de tabelas (DataFrames)

# -----------------------------
# 🧪 Visualização de dados
# -----------------------------
import seaborn as sns  # gráficos estatísticos
import matplotlib.pyplot as plt  # gráficos gerais
from graphviz import Source  # renderização de árvores .dot
import importlib
importlib.reload(plt)  # força recarregamento do matplotlib

# -----------------------------
# ⚙️ Manipulação de caminhos
# -----------------------------
from pathlib import Path  # caminhos multiplataforma (Windows/Linux)

# -----------------------------
# ⚠️ Tratamento de avisos
# -----------------------------
import warnings  # controle de warnings no console
from sklearn.exceptions import ConvergenceWarning  # suprimir avisos do MLPClassifier

# -----------------------------
# 📁 Configurações do projeto
# -----------------------------
from config import (    
    results_dir,     # diretório de saída dos resultados
    algorithms,      # dicionário de classificadores
    cv               # estratégia de validação cruzada
)

# -----------------------------
# ⏱️ Medição de tempo
# -----------------------------
from timeit import default_timer as timer  # cronômetro

# -----------------------------
# 🤖 Classificadores e Modelos
# -----------------------------
from sklearn.tree import DecisionTreeClassifier, export_graphviz  # árvore de decisão
from sklearn.neighbors import KNeighborsClassifier  # kNN
from sklearn.svm import SVC  # máquinas de vetor de suporte
from sklearn.naive_bayes import GaussianNB  # Naive Bayes
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier  # ensembles
from sklearn.neural_network import MLPClassifier  # rede neural
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis  # QDA
from sklearn.base import clone  # duplicação de modelos

# -----------------------------
# 🔀 Pré-processamento e validação
# -----------------------------
from sklearn.preprocessing import StandardScaler  # normalização
from sklearn.model_selection import (
    train_test_split,  # divisão treino/teste
    cross_val_score    # validação cruzada
)

# -----------------------------
# 📈 Avaliação de desempenho
# -----------------------------
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,  # métricas principais
    confusion_matrix, classification_report,  # matriz e relatório de classificação
    roc_curve, auc  # curva e área sob a curva ROC
)

# -----------------------------
# 📉 Redução de dimensionalidade
# -----------------------------
from sklearn.decomposition import PCA  # análise de componentes principais

# -----------------------------
# 🧪 Geração de dados sintéticos
# -----------------------------
from sklearn.datasets import make_moons, make_circles, make_classification  # datasets de brinquedo

# -----------------------------
# 🔍 Visualização de fronteiras
# -----------------------------
from sklearn.inspection import DecisionBoundaryDisplay  # visualização de fronteiras de decisão

## -------------------- 02. Carregamento do Dataset -------------------- 

Esta etapa realiza o carregamento do conjunto de dados a partir de um arquivo `.csv`. Essa é a primeira etapa prática do pipeline de machine learning, preparando os dados para o pré-processamento e análise.

In [ ]:
# Define o diretório onde está o arquivo .csv
db_dir = './db'

# Define o nome do arquivo .csv com os dados (pode ser alterado)
db_file = 'db.csv' 

# Lê o arquivo CSV e armazena em um DataFrame
df = pd.read_csv(f"{db_dir}/{db_file}")

# Exibe as primeiras linhas do DataFrame
df.head()

## -------------------- 03. Conversão de Gênero para Valores Numéricos --------------------

Essa função executa o pré-processamento da variável alvo `gender`, convertendo os valores categóricos ('f' e 'm') para valores numéricos (0 e 1), que são aceitos pelos algoritmos de aprendizado de máquina.

In [ ]:
# Converte coluna 'gender' de 'f'/'m' para 0/1
def convert_gender_to_numeric(df):
    df['gender'] = df['gender'].replace({'f': 0, 'm': 1})  # troca 'f' por 0 e 'm' por 1
    return df  # retorna o DataFrame com a coluna modificada

## -------------------- 04. Visualização da Dispersão entre Portadoras (Pairplot) -------------------- 

Esta etapa realiza uma análise visual dos dados usando pairplot para verificar a separabilidade entre os gêneros com base nas portadoras. Essa visualização auxilia na etapa de exploração de dados (EDA), ajudando a entender como as variáveis se distribuem e se há padrões visuais úteis para o modelo.

In [ ]:
# Importa bibliotecas para visualização e manipulação de arquivos
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

# Cria o diretório 'results' se ainda não existir
Path("results").mkdir(exist_ok=True)

# Define as colunas das portadoras
portadoras = ['rpi1_sc-118', 'rpi1_sc-111', 'rpi1_sc2', 'rpi1_sc32',
              'rpi1_sc67', 'rpi1_sc106', 'rpi1_sc120', 'rpi1_sc121']

# Cria uma cópia do DataFrame com apenas as portadoras e o gênero
df_plot = df[portadoras + ['gender']].copy()

# Converte os valores 0 e 1 para 'Men' e 'Women' (para legenda)
df_plot['gender'] = df_plot['gender'].map({0: 'Women', 1: 'Men'})

# Cria o gráfico de dispersão múltipla (pairplot)
plot = sns.pairplot(df_plot, hue='gender', palette='Set1', corner=True)

# Define o título da figura
plt.suptitle("Carrier Dispersion by Gender", y=1.02)

# Salva o gráfico como imagem
plot.savefig("results/pair_plot.png", dpi=300)
plt.close()

# Mensagem de confirmação
print("Pairplot saved in: results/pair_plot.png")

## -------------------- 05. Avaliação com Validação Cruzada para Todos os Algoritmos -------------------- 

Esta função realiza a avaliação de todos os algoritmos definidos na configuração utilizando validação cruzada estratificada. Essa etapa pertence à fase de avaliação de modelos no pipeline de aprendizado de máquina e permite comparar o desempenho dos classificadores de forma consistente.

In [ ]:
# Roda cross_val_score para todos os algoritmos
def generate_cross_val_score_all_alg(df, ycla):
    print("Starting Assessment.")  # mensagem de início

    result = {}  # dicionário para armazenar os resultados

    # percorre todos os algoritmos definidos no dicionário algorithms
    for alg, clf in algorithms.items():
        print(f"Processing {alg}...")  # exibe o nome do algoritmo atual

        try:
            # executa a validação cruzada e armazena os resultados
            result[alg] = cross_val_score(clf, df, ycla, cv=cv)
        except Exception as error:
            # trata e exibe erros que ocorrerem durante a validação
            print(f'Erro in cross validation. \nErro: {error}')

        print("Done.")  # indica que finalizou o algoritmo atual

    print("Assessment Completed")  # mensagem final

    # retorna os resultados como DataFrame
    return pd.DataFrame.from_dict(result)

## -------------------- 06. Geração e Armazenamento dos Resultados com Validação Cruzada -------------------- 

Esta função realiza o pré-processamento do dataset, executa a avaliação dos algoritmos com validação cruzada e salva os resultados estatísticos (média ± desvio padrão) e o gráfico boxplot no diretório especificado. Ela faz parte da fase de avaliação e documentação dos resultados do pipeline de aprendizado de máquina.

In [ ]:
# Gera boxplot e salva resultados do cross_val_score
def generate_results(name, df, results_dir=results_dir):
    print(f"Generating Results for the Base {name}.\n")  # início da avaliação

    df = convert_gender_to_numeric(df)  # converte 'gender' para valores numéricos
    ycla = df.gender  # separa variável alvo
    df = df.drop("gender", axis=1)  # remove a coluna 'gender' dos atributos

    start = timer()  # inicia a contagem de tempo
    result = generate_cross_val_score_all_alg(df, ycla)  # roda a avaliação
    time = timer() - start  # calcula tempo de execução

    db_result_dir = results_dir / name  # define o diretório de saída
    db_result_dir.mkdir(parents=True, exist_ok=True)  # cria o diretório se não existir

    txt_dir = db_result_dir / f"{name}.txt"  # define o caminho do arquivo de texto
    with open(txt_dir, 'w') as file:
        # escreve média ± desvio padrão por algoritmo
        file.write(result.apply(lambda x: "{:.2f} ± {:.2f}".format(x.mean(), x.std())).to_string())
        file.write(f"\nExecution time: {time:.2f} seconds")  # escreve o tempo total

    # cria o gráfico boxplot com os resultados
    plt.boxplot([scores for scores in result.values()])
    plt.xticks(1 + np.arange(result.shape[1]), result.columns)
    plot_file = db_result_dir / f"{name}_boxplot.png"  # define o nome do arquivo da imagem
    plt.savefig(plot_file, dpi=300)  # salva a imagem
    plt.close()  # fecha a figura

    print(f"Saved Information - Fit Time: {time:.2f}.\n")  # fim da execução

## -------------------- 07. Funções Auxiliares de Avaliação e Visualização --------------------

Este bloco define três funções auxiliares utilizadas durante a avaliação dos modelos: o cálculo das métricas (acurácia, precisão, recall e F1-score), a geração de gráficos boxplot dos resultados de validação cruzada e a visualização da matriz de confusão para cada experimento. Essas funções fazem parte da etapa de avaliação quantitativa e visual no pipeline de aprendizado de máquina.

In [ ]:
# Calcula métricas de avaliação
def generate_scores(y_test, y_pred):
    return [
        accuracy_score(y_test, y_pred),  # calcula acurácia
        precision_score(y_test, y_pred, zero_division=0),  # calcula precisão
        recall_score(y_test, y_pred, zero_division=0),  # calcula recall
        f1_score(y_test, y_pred, zero_division=0)  # calcula f1-score
    ]

# Plota boxplot com os resultados de acurácia
def plot_boxplot(result: dict, alg_name, boxplot_file, y_label_text="Score"):
    df_result = pd.DataFrame.from_dict(result)  # transforma o dicionário em DataFrame

    # Garante que o rótulo do eixo Y seja uma string
    if not isinstance(y_label_text, str):
        print(f"Warning: y_label_text era {type(y_label_text)}. Convertendo.")
        y_label_text = str(y_label_text)

    plt.style.use('default')  # aplica o estilo padrão do matplotlib
    plt.figure(figsize=(8, 5))  # define o tamanho da figura
    plt.boxplot([scores for scores in df_result.values.T])  # plota os boxplots
    plt.title("Leave-One-Experiment-Out Boxplot")  # título do gráfico
    plt.ylabel(y_label_text)  # rótulo do eixo Y
    plt.xticks(1 + np.arange(df_result.shape[1]), df_result.columns)  # define os rótulos no eixo X
    plt.savefig(boxplot_file, dpi=300)  # salva a imagem no caminho especificado
    plt.close()  # fecha o gráfico
    print(f"Boxplot salvo em: {boxplot_file}")  # mensagem de confirmação

# Plota a matriz de confusão para um experimento
def plot_heatmap(Y_test, Y_pred, alg_name, db_result_dir, index):
    cm = confusion_matrix(Y_test, Y_pred, labels=[0, 1])  # gera matriz de confusão 2x2

    plt.figure(figsize=(10, 8))  # define o tamanho da figura
    sns.heatmap(cm, xticklabels=["Female", "Male"], yticklabels=["Female", "Male"],
                cmap="YlGnBu", annot=True, fmt="d")  # plota a matriz com cores e valores
    sns.set(font_scale=1.1)  # ajusta o tamanho da fonte
    plt.xlabel("Predicted Classes")  # rótulo eixo X
    plt.ylabel("True Classes")  # rótulo eixo Y
    plt.title(f"Confusion Matrix - {alg_name}")  # título do gráfico
    plot_file = db_result_dir / f"{alg_name}_confusion_matrix_experiment{index+1}.png"  # caminho de saída
    plt.savefig(plot_file, dpi=300)  # salva a imagem
    plt.close()  # fecha o gráfico

## -------------------- 08. Função para Formatação de Resultados Estatísticos --------------------

Esta função auxilia na apresentação dos resultados, formatando a média e o desvio padrão de uma métrica de avaliação (como acurácia, precisão, etc.) em uma string padronizada. É utilizada na geração dos arquivos de saída para facilitar a leitura dos resultados.

In [ ]:
# Formata string de média ± desvio padrão
def format_string(valor, scores):
    return f"\Mean {valor}: {np.mean(scores):.4f} +- {np.std(scores):.4f}\n"  # retorna string formatada com média e desvio padrão

## -------------------- 09. Avaliação Leave-One-Experiment-Out (LOEO) para um Modelo -------------------- 

Esta função realiza a avaliação de um modelo usando a técnica Leave-One-Experiment-Out (LOEO), onde o conjunto de dados é dividido em 10 blocos (experimentos). Em cada iteração, um bloco é usado como teste e os demais como treino. Para cada rodada, o modelo é treinado, testado e suas métricas armazenadas. Ao final, gera gráficos, salva as métricas em arquivo e retorna os resultados consolidados. Esta etapa pertence à fase de avaliação robusta no pipeline de aprendizado de máquina.

In [ ]:
# Avaliação leave-one-experiment-out para um modelo
def leave_one_experiment_out_evaluation(df, best_alg_name, results_dir, name):
    print(f"\nStarting leave-one-experiment-out evaluation with the best model: {best_alg_name}")  # início da avaliação

    n_experimentos = 10  # total de blocos de experimentos
    tamanho_exp = 400  # tamanho de cada bloco
    scores = {"Accuracy": [], "Precision": [], "Recall": [], "F1-Score": []}  # dicionário para guardar métricas
    y_true_all, y_pred_all = [], []  # listas para acumular rótulos reais e preditos

    db_result_dir = results_dir / name  # define pasta de saída
    db_result_dir.mkdir(parents=True, exist_ok=True)  # cria pasta se não existir
    boxplot_file = db_result_dir / f"{name}_leave_one_exp_boxplot.png"  # caminho do gráfico final

    # loop sobre os 10 experimentos
    for i in range(n_experimentos):
        ini, fim = i * tamanho_exp, (i + 1) * tamanho_exp  # define índice do bloco de teste
        test = df.iloc[ini:fim].copy()  # extrai bloco de teste
        train = pd.concat([df.iloc[:ini], df.iloc[fim:]]).copy()  # junta os blocos de treino

        train = convert_gender_to_numeric(train)  # converte 'gender' em treino
        test = convert_gender_to_numeric(test)  # converte 'gender' em teste

        y_train = train.gender  # separa variável alvo de treino
        X_train = train.drop("gender", axis=1)  # separa atributos de treino
        y_test = test.gender  # variável alvo de teste
        X_test = test.drop("gender", axis=1)  # atributos de teste

        model = clone(algorithms[best_alg_name])  # clona o melhor modelo
        model.fit(X_train, y_train)  # treina o modelo
        y_pred = model.predict(X_test)  # realiza a predição

        acc, precision, recall, f1 = generate_scores(y_test, y_pred)  # calcula métricas
        scores["Accuracy"].append(acc)  # armazena acurácia
        scores["Precision"].append(precision)  # armazena precisão
        scores["Recall"].append(recall)  # armazena recall
        scores["F1-Score"].append(f1)  # armazena f1-score

        y_true_all.extend(y_test)  # acumula rótulos reais
        y_pred_all.extend(y_pred)  # acumula predições

        print(f"Experment {i+1}: Accuracy = {acc:.4f}")  # exibe acurácia do experimento
        plot_heatmap(y_test, y_pred, best_alg_name, db_result_dir, i)  # plota matriz de confusão

    # Plota boxplot com as acurácias dos 10 experimentos
    plot_boxplot({"Accuracy": scores["Accuracy"]}, best_alg_name, boxplot_file, "Accuracy")

    # Salva os resultados em arquivo de texto
    txt_file = db_result_dir / f"{name}_leave_one_exp_scores.txt"
    with open(txt_file, 'w') as f:
        f.write("Accuracies leave-one-experiment-out:\n")
        for i, score in enumerate(scores["Accuracy"]):
            f.write(f"Experiment {i+1}: {score:.4f}\n")
        for key, score in scores.items():
            f.write(format_string(key, score))

    print(f"Scores saved in: {txt_file}\n")  # confirmação
    return scores  # retorna o dicionário de métricas

## -------------------- 10. Avaliação Completa para o Melhor Modelo --------------------

Esta função executa o processo completo de avaliação para o melhor modelo encontrado via validação cruzada. Ela carrega os dados, realiza a conversão da variável alvo, identifica o melhor classificador com base na média de desempenho e executa a avaliação Leave-One-Experiment-Out (LOEO) com esse modelo. Esta etapa fecha o ciclo de avaliação automatizada do pipeline de aprendizado de máquina.

In [ ]:
# Gera avaliação completa para o melhor modelo
def generate_one_result(db_name):
    df = pd.read_csv(db_name)  # carrega o dataset do arquivo CSV
    df = convert_gender_to_numeric(df)  # converte 'gender' para 0/1
    ycla = df.gender  # variável alvo
    X = df.drop("gender", axis=1)  # atributos de entrada

    result = generate_cross_val_score_all_alg(X, ycla)  # avalia todos os modelos com cross_val
    best_alg_name = result.mean().idxmax()  # identifica o modelo com maior média de desempenho

    print(f"\nBest Model: {best_alg_name}")  # exibe o nome do melhor modelo

    # executa a avaliação LOEO com o melhor modelo
    leave_one_experiment_out_evaluation(
        pd.read_csv(db_name),  # carrega novamente o CSV original
        best_alg_name,  # nome do melhor modelo
        results_dir,  # pasta de saída
        f"{db_name}_{best_alg_name}"  # nome do diretório de saída
    )

    # mensagem final com caminho de onde os resultados foram salvos
    print(f"Best Model: {best_alg_name} saved in: {results_dir / f'{db_name}_{best_alg_name}'}")

## -------------------- 11. Avaliação Leave-One-Experiment-Out para Todos os Modelos --------------------

Esta função executa a avaliação LOEO para todos os algoritmos definidos na configuração. Cada modelo é avaliado separadamente, e as acurácias de cada experimento são armazenadas. Ao final, um gráfico boxplot é gerado comparando os desempenhos entre os modelos. Esta etapa pertence à fase de comparação e análise de desempenho entre os classificadores no pipeline de aprendizado de máquina.

In [ ]:
from pathlib import Path  # biblioteca para manipulação de caminhos

# Executa avaliação leave-one-out para todos os modelos disponíveis
def leave_one_experiment_out_evaluation_all_models(db_name):
    print("Leave-one-experiment-out evaluation completed for all models.")  # início do processo

    acc_results = {}  # dicionário para guardar resultados de acurácia por modelo

    # percorre todos os algoritmos definidos
    for alg_name in algorithms.keys():
        # Gera nome de pasta/arquivo sem extensões estranhas (ex: .csv ou ./)
        name = Path(db_name).stem.replace(".", "_") + f"_{alg_name}"

        df = pd.read_csv(db_name)  # carrega o dataset

        # Executa avaliação LOEO para o modelo atual
        scores = leave_one_experiment_out_evaluation(df, alg_name, results_dir, name)

        # Armazena apenas as acurácias
        acc_results[alg_name] = scores["Accuracy"]

    # Gera gráfico comparativo das acurácias entre os modelos
    boxplot_file = results_dir / "acc_boxplot.png"
    plot_boxplot(acc_results, "Modelos", boxplot_file, "Accuracy")

    print("Leave-one-experiment-out evaluation completed for all models.")  # fim

## -------------------- 12. Execução Final da Avaliação para Todos os Modelos --------------------

Este bloco define o caminho do dataset `.csv` e executa a função de avaliação Leave-One-Experiment-Out para todos os modelos disponíveis. Ao final, imprime os arquivos gerados na pasta `results/`, encerrando a fase principal de avaliação do pipeline de aprendizado de máquina.

In [ ]:

# Caminho completo para o dataset
db_name = f"{db_dir}/{db_file}"

# Executa a avaliação LOEO para todos os modelos
leave_one_experiment_out_evaluation_all_models(db_name)

# Lista os arquivos gerados na pasta de resultados
print("Results generated in:", os.listdir('./results'))


## -------------------- 13. Relatório Final com Todos os Dados Usando o Melhor Modelo --------------------

Esta etapa realiza a avaliação final do melhor modelo identificado pela média de acurácia na validação cruzada. O modelo é treinado com todos os dados disponíveis e, em seguida, avaliado no mesmo conjunto para fins descritivos. São geradas as métricas detalhadas (precisão, recall, f1-score e suporte) e a matriz de confusão final, que mostram como o modelo se comporta globalmente. A matriz é salva em `results/melhor_modelo_confusion_matrix_FINAL.png`.

In [ ]:
# Garante que os dados estejam prontos
df_full = convert_gender_to_numeric(df.copy())  # Converte coluna 'gender' de string para número
X = df_full.drop("gender", axis=1)              # Separa atributos preditores
y = df_full["gender"]                           # Separa rótulo

# Gera cross_val_score para todos os modelos e identifica o melhor
avaliacoes = generate_cross_val_score_all_alg(X, y)        # Avaliação cruzada
melhor_algoritmo = avaliacoes.mean().idxmax()              # Seleciona o melhor classificador
print(f"Melhor modelo geral com base na média do cross_val_score: {melhor_algoritmo}")

# Clona e treina o melhor modelo com todos os dados
melhor_modelo = clone(algorithms[melhor_algoritmo])        # Clona pipeline
melhor_modelo.fit(X, y)                                    # Treina com todos os dados

# Realiza predição nos mesmos dados (avaliação descritiva)
y_pred = melhor_modelo.predict(X)

# Gera e imprime relatório de classificação
relatorio = classification_report(y, y_pred, target_names=["Women", "Men"], digits=4)
print("\nClassification Report (all datas):")
print(relatorio)

# Salva o relatório em um arquivo TXT
Path("results").mkdir(exist_ok=True)
with open("results/relatorio_final.txt", "w") as f:
    f.write("Classification Report (all datas):\n\n")
    f.write(relatorio)

# Matriz de confusão final
cm = confusion_matrix(y, y_pred, labels=[0, 1])
plt.figure(figsize=(8, 6))
sns.heatmap(cm,
            annot=True,
            fmt="d",
            cmap="Blues",
            xticklabels=["Women", "Men"],
            yticklabels=["Women", "Men"])
plt.xlabel("Predicted Class")
plt.ylabel("True Class")
plt.title(f"Final Confusion Matrix - {melhor_algoritmo}")

# Salva a imagem da matriz de confusão
plt.savefig("results/melhor_modelo_confusion_matrix_FINAL.png", dpi=300)
plt.close()
print("Final COnfusion Matrix saved in: results/melhor_modelo_confusion_matrix_FINAL.png")


## -------------------- 14. Comparação Visual entre Classificadores --------------------

Adicionalmente, é gerado um gráfico com a comparação visual dos limites de decisão de vários algoritmos de classificação em datasets sintéticos. Esse gráfico ilustra a capacidade de generalização e a forma como cada modelo segmenta o espaço de decisão. Faz parte da etapa de análise comparativa/visualização.

In [ ]:
# Classificadores utilizados no projeto
names = [
    "kNN", "Decision Tree", "Big Tree", "Naive Bayes", "SVM Linear", "SVM RBF"
]

classifiers = [
    KNeighborsClassifier(3),  # kNN
    DecisionTreeClassifier(max_depth=5),  # Árvore pequena
    DecisionTreeClassifier(),  # Árvore grande
    GaussianNB(),  # Naive Bayes
    SVC(kernel="linear", C=0.025, probability=True),  # SVM Linear
    SVC(gamma=2, C=1, probability=True),  # SVM RBF
]

# Datasets sintéticos para visualização
datasets = [
    make_moons(noise=0.3, random_state=0),
    make_circles(noise=0.2, factor=0.5, random_state=1),
    make_classification(
        n_features=2, n_redundant=0, n_informative=2,
        random_state=42, n_clusters_per_class=1
    )
]

# Ignora avisos de convergência (MLP, etc)
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)

    # Cria figura com várias subplots
    figure = plt.figure(figsize=(20, 6))
    i = 1

    for ds in datasets:
        X, y = ds
        X = StandardScaler().fit_transform(X)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=42)

        # Primeira coluna: dados originais
        ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
        ax.set_title("Input data")
        ax.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.coolwarm, edgecolors='k')
        i += 1

        # Colunas seguintes: classificadores
        for name, clf in zip(names, classifiers):
            ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
            clf.fit(X_train, y_train)
            score = clf.score(X_test, y_test)

            # Exibe fronteira de decisão
            display = DecisionBoundaryDisplay.from_estimator(
                clf, X, response_method="predict", cmap=plt.cm.coolwarm, alpha=0.8, ax=ax
            )
            ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=plt.cm.coolwarm, edgecolors='k')
            ax.set_title(f"{name}\nAccuracy: {score:.2f}")
            i += 1

    plt.tight_layout()
    plt.savefig("results/classifier_comparison.png", dpi=300)
    plt.close()
    print("Comparison Graph saved in: results/classifier_comparison.png")

## -------------------- 15. Geração da Árvore de Decisão Geral com Todos os Dados --------------------

Este bloco treina uma árvore de decisão com todos os dados disponíveis e gera uma representação visual completa da estrutura da árvore. Esta etapa complementa a avaliação, permitindo uma interpretação visual do modelo final e das variáveis mais relevantes, sendo parte da explicação e apresentação de resultados do projeto.

In [ ]:
# Garante que os dados estejam prontos
df_full = convert_gender_to_numeric(df.copy())  # converte gênero para 0 e 1
X = df_full.drop("gender", axis=1)  # separa atributos
y = df_full["gender"]  # variável alvo

# Treina o modelo completo com todos os dados
modelo_geral = DecisionTreeClassifier(
    max_depth=5,  # profundidade máxima da árvore
    criterion='entropy',  # critério de divisão
    random_state=42  # semente para reprodutibilidade
)
modelo_geral.fit(X, y)  # treina a árvore com todos os dados

# Garante que a pasta 'results' exista
Path("results").mkdir(exist_ok=True)  # cria a pasta se não existir

# Exporta a árvore para um arquivo .dot
export_graphviz(
    modelo_geral,
    out_file="results/credit_tree_geral.dot",  # caminho do arquivo de saída
    feature_names=X.columns,  # nomes das variáveis
    class_names=["Women", "Men"],  # nomes das classes
    rounded=True,  # cantos arredondados
    filled=True  # cores nas folhas
)

# Renderiza o arquivo .dot e salva como .png
graph = Source.from_file("results/credit_tree_geral.dot")  # carrega o arquivo .dot
graph.render("results/credit_tree_geral", format="png", cleanup=True)  # salva como imagem

# Mensagem final de confirmação
print("General Tree saved in: results/credit_tree_geral.png")

## -------------------- 16. Curva ROC e Visualização PCA dos Dados --------------------

Este bloco complementa a análise dos resultados com dois gráficos. Primeiro, a Curva ROC (Receiver Operating Characteristic), que mostra a performance do modelo ao variar o limiar de decisão, junto com o valor da AUC (Área sob a curva). Em seguida, um gráfico de dispersão usando PCA (Análise de Componentes Principais), reduzindo os dados para 2 dimensões e colorindo por gênero, permitindo visualizar possíveis separações entre as classes.

In [ ]:
# Função para plotar a curva ROC com AUC
def plot_roc_curve(model, X_test, y_test, label="Modelo"):
    # Verifica se o modelo possui método predict_proba ou decision_function
    if hasattr(model, "predict_proba"):
        probs = model.predict_proba(X_test)[:, 1]  # usa probabilidade da classe positiva
    else:
        probs = model.decision_function(X_test)  # usa função de decisão (ex: SVM)

    # Calcula os valores de FPR (falsos positivos) e TPR (verdadeiros positivos)
    fpr, tpr, _ = roc_curve(y_test, probs)

    # Calcula a área sob a curva (AUC)
    roc_auc = auc(fpr, tpr)

    # Cria o gráfico
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'{label} (AUC = {roc_auc:.2f})')  # curva principal
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')  # linha de referência (aleatório)
    plt.xlabel('FPR (False Positives)')  # eixo X
    plt.ylabel('TPR (True Positives)')  # eixo Y
    plt.title('ROC Curve')  # título
    plt.legend(loc="lower right")  # legenda
    plt.grid()  # adiciona grade
    plt.savefig("results/roc_curve.png", dpi=300)  # salva o gráfico
    print("ROC Curve saved in: results/roc_curve.png") 
    plt.close()  # fecha figura

# Plota a curva ROC para o modelo geral
plot_roc_curve(modelo_geral, X, y, label="General Tree")

# Reduz os dados para 2D com PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)  # aplica a transformação PCA

# Converte rótulos numéricos para texto
labels = y.map({0: 'Women', 1: 'Men'})

# Cria gráfico de dispersão por gênero
plt.figure(figsize=(8, 6))
for label in labels.unique():
    idx = labels == label
    plt.scatter(X_pca[idx, 0], X_pca[idx, 1], label=label, alpha=0.7)  # plota cada grupo

plt.title("PCA Dispersion by Gender - 8 subcarriers")  # título do gráfico
plt.xlabel("Principal Component 1")  # eixo X
plt.ylabel("Principal Component 2")  # eixo Y
plt.legend()  # exibe legenda
plt.savefig("results/pca_2d_scatter.png", dpi=300)  # salva o gráfico
print("PCA 2D Scatter saved in: results/pca_2d_scatter.png") 
plt.close()  # fecha a figura

## -------------------- 17. PCA com Visualização Bidimensional (8 e 271 subportadoras) --------------------

Essa etapa realiza uma redução de dimensionalidade via PCA para facilitar a visualização dos dados em 2D. São usados dois conjuntos diferentes (um com 271 subportadoras e outro com 8), permitindo comparação visual entre os gêneros após a transformação.

In [ ]:
# === PCA - Visualização com 271 subportadoras ===

file_path = './db/db_400.csv'  # caminho do CSV com 271 subportadoras
df = pd.read_csv(file_path)  # carrega o CSV como DataFrame

y = df.iloc[:, 0]  # separa a variável alvo (0 = mulher, 1 = homem)
X = df.iloc[:, 1:]  # separa os atributos (colunas de subportadoras)

scaler = StandardScaler()  # instancia normalizador (zero média, variância 1)
X_scaled = scaler.fit_transform(X)  # aplica normalização aos dados

labels = y.map({0: 'Men', 1: 'Women'})  # converte rótulos numéricos em texto

pca = PCA(n_components=2)  # define PCA com 2 componentes principais
X_pca = pca.fit_transform(X_scaled)  # reduz os dados para 2D

plt.figure(figsize=(8, 6))  # define tamanho da figura
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1],
                hue=y.map({0: 'Women', 1: 'Men'}), palette='Set1')  # cria gráfico colorido por gênero
plt.title('PCA Dispersion by Gender - 271 subcarriers')  # título do gráfico
plt.xlabel("Principal Component 1")  # legenda eixo X
plt.ylabel("Principal Component 2")  # legenda eixo Y
plt.legend()  # exibe legenda
plt.grid(True)  # exibe grade no gráfico
plt.tight_layout()  # ajusta margens
plt.savefig("results/pca_271p.png", dpi=300)  # salva a imagem no diretório
print("PCA 271p: results/pca_271p.png")
plt.close()  # fecha a figura para liberar memória


# === PCA - Visualização com 8 subportadoras ===

file_path = './db/db.csv'  # caminho do CSV com 8 subportadoras
df = pd.read_csv(file_path)  # carrega o CSV como DataFrame

y = df.iloc[:, 0]  # variável alvo (0 = mulher, 1 = homem)
X = df.iloc[:, 1:]  # atributos

scaler = StandardScaler()  # normalizador padrão
X_scaled = scaler.fit_transform(X)  # aplica normalização

labels = y.map({0: 'Men', 1: 'Women'})  # transforma 0/1 em texto

pca = PCA(n_components=2)  # PCA para 2 dimensões
X_pca = pca.fit_transform(X_scaled)  # aplica a transformação PCA

plt.figure(figsize=(8, 6))  # tamanho da figura
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1],
                hue=y.map({0: 'Women', 1: 'Men'}), palette='Set1')  # gráfico com cores por gênero
plt.title('PCA Dispersion by Gender - 8 subcarriers')  # título do gráfico
plt.xlabel("Principal Component 1")  # eixo X
plt.ylabel("Principal Component 2")  # eixo Y
plt.legend()  # legenda
plt.grid(True)  # exibe grade
plt.tight_layout()  # ajusta layout
plt.savefig("results/pca_8p.png", dpi=300)  # salva imagem no diretório results
print("PCA 8p: results/pca_8p.png")
plt.close()  # fecha a figura

## -------------------- Fim do código --------------------